# Running Continuity on AWS from a Jupyter Notebook

Outline: 

1. [Step 1: Launch an AWS Instance](#Step_1)
1. [Step 2: Define Continuity Simulation Parameters in a Python Script](#Step_2)
1. [Step 3: Move files to the EC2 instance](#Step_3)
1. [Step 4: Run Continuity simulation](#Step_4)
1. [Step 5: Copy files back from EC2 instance and terminate the instance](#Step_5)
1. [Step 6: Terminate the instance](#Step_6)

<a id="Step_1"></a>
## Step 1: Launch an AWS Instance

#### Input Parameters
The user needs to provide their access key information, a private key, and define the properties for their instance.  All of these can be generated through python or with AWS CLI, but they may be easier to do through the aws console if only a single account is being set up.

In [ ]:
import boto3
import subprocess
import sys

In [ ]:
# Access keys - These can be set up by creating a new IAM account for each user
aws_access_key_id = ''
aws_secret_access_key = ''

# Key pair - The key name and path to the private k
KeyName = ''
priv_key = ''

# Instance properties - ensure that the image used has Continuity pre-installed
region = 'us-west-1'
ImageId = 'ami-0948262f83c23b44c'
InstanceType = 'c5.large'
NameTag = 'jupyter_test'
VolumeSize = 12   # GB; current Continuity AMI fills up the entire 8GB volume but could be fixed

#### Launch instance

In [ ]:
ec2 = boto3.resource('ec2', region,
        aws_access_key_id = aws_access_key_id,
        aws_secret_access_key = aws_secret_access_key)

In [ ]:
#Tagging the isntance with a name is not needed, but it can be useful
TagSpec = [{'ResourceType':'instance','Tags': [{
                    'Key': 'Name',
                    'Value': NameTag}]}]

In [ ]:
instances = ec2.create_instances(ImageId=ImageId, 
                                InstanceType=InstanceType, 
                                KeyName=KeyName, 
                                TagSpecifications=TagSpec,
                                BlockDeviceMappings=[{"DeviceName": "/dev/sda1","Ebs" : { "VolumeSize" : VolumeSize }}],
                                IamInstanceProfile={'Name': 'SSMInstanceProfile'},
                                MinCount=1,
                                MaxCount=1)
instance = instances[0]
instance.wait_until_running()
instance.load()

#### Check that the instance is running

The above cell will not complete until the instance is actually running - `instane.wait_until_running()`.  We can double check that it is running by listing all currently running instances for the CMRG account (on the given region)

In [ ]:
#Functions could be moved to a remote script to make the notebook cleaner
def show_instance(status):
    instances = ec2.instances.filter(
        Filters=[{'Name': 'instance-state-name', 'Values': [status]}])
    for instance in instances:
        print(instance.tags[0]['Value'], instance.id, instance.instance_type, instance.image_id, instance.public_ip_address)

In [ ]:
show_instance('running')

<a id="Step_2"></a>
## Step 2: Define Continuity Simulation Parameters in a Python Script

Running Continuity on AWS through a Jupyter Notebook requires using a python run script to execute your Continuity simulation.  It is easiest to run from a template for the class of problem you are looking to run and using the Jupyter Notebook to make edits to that file.  This section will be different for every specific use case.

For this example, I am going to change the conductivity and the simulation name. The location of the .cont6 file on the EC2 instance is also needed
#### Input Parameters

In [ ]:
cont6_file = './sheet2304.cont6'
run_script = './sheet_template.py'

ec2_path = '/home/ubuntu/sims/'

run_name = 'sheetStim1'
conductivity = '0.1'

#### Edit Python File

In [ ]:
# Open the template run script
tmp = open(run_script,'r')
tmp_list = tmp.readlines()
tmp.close()

# Edit the lines defining the run name and the conductivity
tmp_list[1] = 'fName = \'%s\'\n'%run_name
tmp_list[2] = 'f11 = %s\n'%conductivity
tmp_list[3] = 'cont6path = \'%s\'\n'%(ec2_path+cont6_file[2:])

# Resave the run script
outFile = open(run_name+'.py','w')
outFile.writelines(tmp_list)
outFile.close()

<a id="Step_3"></a>
## Step 3: Move files to the EC2 instance

Here we need to provide the path location to put the files on the EC2 instance.  In the AMI used here, there is a folder named '~/sims/' to place scripts in and run simulations

#### Copy files to instance

In [ ]:
# First define a function using subprocess to call scp
# Note that the StrictHostKeyChecking=accept-new is not ideal. More native AWS methods may be better or perhaps paramiko
def copy_to_remote(path_to_pem, local_file_path, host_username, host_ip, host_path):
        scp = subprocess.Popen(['scp', '-i', path_to_pem, '-o', 'StrictHostKeyChecking=accept-new', local_file_path, '%s@%s:%s'%(host_username, host_ip, host_path)],
                       shell=False,
                       stdout=subprocess.PIPE,
                       stderr=subprocess.PIPE)
        sts = scp.wait()
        error = scp.stderr.readlines()
        result = scp.stdout.readlines()
        if error != []:
                print(sys.stderr, "ERROR.: %s"%error)
                return "error"
        else:
                return print('success')

In [ ]:
# Move python run script
copy_to_remote(priv_key, run_name+'.py', 'ubuntu', instance.public_ip_address, ec2_path)

# Move .cont6 file
copy_to_remote(priv_key, cont6_file, 'ubuntu', instance.public_ip_address, ec2_path)

<a id="Step_4"></a>
## Step 4: Run Continuity simulation

In [ ]:
ec2_cont_path = '/home/ubuntu/Continuity/'

In [ ]:
ssm_client = boto3.client('ssm', region,
        aws_access_key_id = aws_access_key_id,
        aws_secret_access_key = aws_secret_access_key)
#ssm_client.describe_instance_information()

In [ ]:
def remote_exec(client, commands, instance_ids):
    resp = client.send_command(
        DocumentName="AWS-RunShellScript", # One of AWS' preconfigured documents
        Parameters={'commands': commands},
        InstanceIds=instance_ids,
    )
    return resp

In [ ]:
commands = [ec2_cont_path + 'continuity' \
            + ' --unbuffered --full --no-threads --batch ' \
            + ec2_path + run_name+'.py']# \
#            + ' 2>&1 > '+ ec2_path+run_name+'.log']
resp = remote_exec(ssm_client,commands,[instance.id])

In [ ]:
command_id = resp['Command']['CommandId']
output = ssm_client.get_command_invocation(
      CommandId=command_id,
      InstanceId=instance.id,
    )
print(output)

<a id="Step_5"></a>
## Step 5: Copy files back from EC2 instance and terminate the instance

#### Input Parameters

The final input is a local path to copy the files to

In [ ]:
local_path = './'
ec2_working_dir = '/home/ubuntu/.continuity/working/'
output_file = 'AP_out_'+run_name+'.txt'

#### Copy Continuity output files

In [ ]:
def copy_from_remote(path_to_pem, host_username, host_ip, host_path, local_file_path):
        scp = subprocess.Popen(['scp', '-i',  path_to_pem, '-o', 'StrictHostKeyChecking=accept-new', '%s@%s:%s'%(host_username, host_ip, host_path), local_file_path],
                       shell=False,
                       stdout=subprocess.PIPE,
                       stderr=subprocess.PIPE)
        sts = scp.wait()
        error = scp.stderr.readlines()
        result = scp.stdout.readlines()
        if error != []:
                print(sys.stderr, "ERROR: %s"%error)
                return "error"
        else:
                return print('success')

In [ ]:
# Retrieve AP_out file
copy_from_remote(priv_key, 'ubuntu', instance.public_ip_address, ec2_working_dir+output_file, local_path)

# Retrieve log file
copy_from_remote(priv_key, 'ubuntu', instance.public_ip_address, ec2_path+run_name+'.log', local_path)

<a id="Step_6"></a>
# Step 6: Terminate the instance
Terminating the instance cannot be undone.  If you just want to stop the instance and may want to come back to it later, it is better to stop it.  By default, the memory will also be cleared so any files remaining on the instance that were not copied over will be lost.

In [ ]:
instance.terminate()

In [ ]:
show_instance('running')